# Задание 3.

Сдаем ноутбук, где будет извлечение всех концептов, которые сможете, и сам итоговый файл в формате, который указан в конце ноутбука занятия.

Описание задания: Когда пациент принимает какое-либо лекарство, у него может возникнуть побочный эффект от лекарства или побочную реакцию на лекарство (ADR). Мы можем использовать CUI, уникальный идентификатор концепта для Metathesaurus, чтобы определить, какие нежелательные реакции упоминаются в сообщениях пациентов.

Цель этого задания – помочь вам научиться идентифицировать (ADR) по описаниям пациентов о нежелательных реакциях на лекарственные препараты.

# Уровни сложности извлечения концептов.

На уровне 1 вы обнаружите, что слова, описывающие нежелательную реакцию, точно такие же, как понятия CUI или могут быть производными одного слова.

> Вот пример:
> - Sentence text: `I was unable to sleep, had blurred vision, and felt sick to my stomach.`
> - ADR: **blurred vision**
> - CUI concept: **blurred vision**
> - CUI: **C0344232** (You can lookup CUI at [SIDER](http://sideeffects.embl.de/))

> - Sentence text: `Muscle spasms, muscle twitching, muscle soreness, insomnia, mental confusion, flush, brain zaps.`
> - ADR: **flush**
> - CUI Concept: **flushing**
> - CUI: **C0016382** (You can lookup CUI at [SIDER](http://sideeffects.embl.de/))

На уровне 2 вы можете обнаружить, что слова, описывающие нежелательную реакцию, совпадают с понятиями CUI, хотя порядок слов может быть другим.
> Вот пример:
> - Sentence text: `Weight gain, HAIR LOSS, increased depression, fatigue, lethargy.`
> - ADR: **HAIR LOSS**
> - CUI concept: **Loss of hair**
> - CUI: **C0002170** (You can lookup CUI at [SIDER](http://sideeffects.embl.de/))

На уровне 3 вы можете обнаружить, что, помимо разницы в порядке слов, только некоторые слова, описывающие нежелательную реакцию, точно такие же, как в понятиях CUI.
> Вот пример:
> - Sentence text: `It's been four days since I discontinued, and I'm still wiped out and dizzy with flu-like symptoms.`
> - ADR: **flu-like symptoms**
> - CUI concept: **Influenza-like symptoms**
> - CUI: **C0392171** (You can lookup CUI at [SIDER](http://sideeffects.embl.de/))

На уровне 4 вы обнаружите, что ни одно из слов, описывающих нежелательную реакцию, не совпадает со словами в концепциях CUI, но при этом имеет то же значение.
> Вот пример:
> - Sentence text: `Also I had a severe inablility to concentrate which made it impossible to do my job,I am a Registered Nurse.`
> - ADR: **impossible to do my job**
> - CUI concept: **Restricted work performance**
> - CUI: **C0557386** (You can lookup CUI at [SIDER](http://sideeffects.embl.de/))

На уровне 5 вы можете обнаружить, что слова, описывающие побочную реакцию, не совпадают с понятиями CUI и имеют разные значения. ADR может предлагать концепцию CUI посредством следствия или посредством какого-либо другого лингвистического или контекстуального механизма. Если вы сумеете их идентифицировать, это будет очень впечатляюще!
> Вот пример:
> - Sentence text: `20 pounds total in 6 months Appetite increase .`
> - ADR: **20 pounds total**
> - CUI concept: **Body Weight Changes**
> - CUI: **C0005911** (You can lookup CUI at [SIDER](http://sideeffects.embl.de/))

Ваш финальный результат представьте в виде csv файла с колонками: sentence_id (например: для первой строки файла
1 extreme weight gain, short-term memory loss, hair loss. sentence_id == drug_ADRs.txt), вторая колонка - CUI, следующие 5 колонок: level_1 .... level_5 для каждого уровня, где будет либо 1 либо 0.

# Импорт библиотек

In [1]:
import pandas as pd

In [2]:
from google.colab import drive

In [3]:
import nltk

In [4]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Загрузка данных

In [8]:
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
concepts_path = 'drive/MyDrive/Применение NLP в Здравоохранении/концепты.tsv'

In [22]:
side_effects_path = 'drive/MyDrive/Применение NLP в Здравоохранении/побочные_эффекты.txt'

In [24]:
concepts_df = pd.read_csv(concepts_path, sep='\t', header=None)
concepts_df.columns = ['CUI', 'Concept', 'SNOMED_CODE']
concepts_df.head()

,CUI,Concept,SNOMED_CODE
0,CUI,CONCEPT,SNOMED_CODE
1,C0000765,"Excessive body weight gain,Excessive weight gain",224994002
2,C0701811,"Poor short-term memory,Poor short-term memory",247592009
3,C0002170,"Alopecia,Loss of hair",278040002
4,C0549622,"Sexual Dysfunction,Sexual disorder",231532002


In [25]:
with open(side_effects_path, 'r', encoding='utf-8') as f:
    side_effects_data = f.readlines()

In [26]:
side_effects_data[:5]

['id\tText\n',
 '1\textreme weight gain, short-term memory loss, hair loss.\n',
 '2\tCOMPLETELY DESTROYED SEXUALLY FUNCTIONING .\n',
 '3\tJust TWO tablets of Lexapro 10mg completely destroyed my sexual functioning, probably for life.\n',
 "4\tIt's called PSSD: post-SSRI sexual dysfunction.\n"]

In [27]:
concepts_df = pd.read_csv(concepts_path, sep='\t', header=None)
concepts_df.head()

,0,1,2
0,CUI,CONCEPT,SNOMED_CODE
1,C0000765,"Excessive body weight gain,Excessive weight gain",224994002
2,C0701811,"Poor short-term memory,Poor short-term memory",247592009
3,C0002170,"Alopecia,Loss of hair",278040002
4,C0549622,"Sexual Dysfunction,Sexual disorder",231532002


In [28]:
num_columns = concepts_df.shape[1]
num_columns

3

In [29]:
concepts_df.columns = ['CUI', 'Concept', 'Details']

In [30]:
concepts_df.head()

,CUI,Concept,Details
0,CUI,CONCEPT,SNOMED_CODE
1,C0000765,"Excessive body weight gain,Excessive weight gain",224994002
2,C0701811,"Poor short-term memory,Poor short-term memory",247592009
3,C0002170,"Alopecia,Loss of hair",278040002
4,C0549622,"Sexual Dysfunction,Sexual disorder",231532002


In [31]:
side_effects_data = []
with open(side_effects_path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        side_effects_data.append((i, line.strip()))

In [32]:
side_effects_data[:5]

[(0, 'id\tText'),
 (1, '1\textreme weight gain, short-term memory loss, hair loss.'),
 (2, '2\tCOMPLETELY DESTROYED SEXUALLY FUNCTIONING .'),
 (3,
  '3\tJust TWO tablets of Lexapro 10mg completely destroyed my sexual functioning, probably for life.'),
 (4, "4\tIt's called PSSD: post-SSRI sexual dysfunction.")]

# Поиск совпадений

In [17]:
from difflib import SequenceMatcher
import csv

In [50]:
from nltk.stem import PorterStemmer

In [60]:
from nltk.corpus import wordnet as wn

In [61]:
def match_levels(concept, text):
    stemmer = PorterStemmer()

    def get_stem_words(words):
        return set(stemmer.stem(word.lower()) for word in words.split())

    def get_synonyms(word):
        synonyms = set()
        for syn in wn.synsets(word):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name().lower())
        return synonyms

    def get_all_concept_words(concept_words):
        all_words = set()
        for word in concept_words:
            all_words.add(word)
            all_words.update(get_synonyms(word))
        return all_words

    concept_words = get_stem_words(concept)
    text_words = get_stem_words(text)

    all_concept_words = get_all_concept_words(concept_words)
    all_text_words = get_all_concept_words(text_words)

    # Уровень 1: Полное совпадение или однокоренные слова
    level_1 = 1 if concept_words == text_words else 0

    # Уровень 2: Совпадение слов, но в другом порядке (если не совпало полностью)
    level_2 = 1 if concept_words == text_words and not level_1 else 0

    # Уровень 3: Частичное совпадение (некоторые слова совпадают)
    level_3 = 1 if concept_words & text_words and not (level_1 or level_2) else 0

    # Уровень 4: Синонимы
    level_4 = 1 if all_concept_words & all_text_words and not (level_1 or level_2 or level_3) else 0

    # Уровень 5: Контекстуальное или следственное совпадение
    level_5 = 0
    if not (level_1 or level_2 or level_3 or level_4):
        # Простая эвристика: если есть хотя бы один синоним, совпадающий с концепцией
        level_5 = 1 if any(wn.synsets(word) for word in text_words) else 0

    return level_1, level_2, level_3, level_4, level_5


In [62]:
output_data = []

In [64]:
# нахождение совпадений
for sid, side_effect in side_effects_data[1:]:  # Skip the header
    for _, row in concepts_df.iterrows():
        cui = row['CUI']
        concept = row['Concept']
        level_1, level_2, level_3, level_4, level_5 = match_levels(concept, side_effect)
        output_data.append([sid, cui, level_1, level_2, level_3, level_4, level_5])


In [65]:
output_csv_path = "drive/MyDrive/Применение NLP в Здравоохранении/side_effects_matching.csv"
with open(output_csv_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["sentence_id", "CUI", "level_1", "level_2", "level_3", "level_4", "level_5"])
    writer.writerows(output_data)

output_csv_path

'drive/MyDrive/Применение NLP в Здравоохранении/side_effects_matching.csv'

In [66]:
df = pd.read_csv('drive/MyDrive/Применение NLP в Здравоохранении/side_effects_matching.csv')

In [69]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
sentence_id,1172840.0,1076.500000,621.229087,1.0,538.75,1076.5,1614.25,2152.0
level_1,1172840.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0
level_2,1172840.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0
level_3,1172840.0,0.060652,0.238691,0.0,0.00,0.0,0.00,1.0
level_4,1172840.0,0.079658,0.270763,0.0,0.00,0.0,0.00,1.0
level_5,1172840.0,0.829633,0.375955,0.0,1.00,1.0,1.00,1.0
